In [1]:
from google.colab import drive

# Connect to Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
folder_path = "/content/drive/My Drive/emails"

In [3]:
!pip install pandas numpy faiss-cpu sentence-transformers
!pip install faiss-cpu sentence-transformers fuzzywuzzy pandas
!pip install --upgrade --quiet  langchain langchain-core langchain-community langchain-text-splitters langchain-milvus
#!pip install openai==0.28
!pip install groq
!pip install tiktoken
!pip install pymilvus langchain
!pip install --upgrade langchain-milvus
!pip install fastapi uvicorn openai pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [4]:
import os
import base64
import pandas as pd
import numpy as np
import faiss
import os
import pandas as pd
from email import policy
from email.parser import BytesParser
from io import BytesIO
from sentence_transformers import SentenceTransformer
import faiss
import nltk
import json
import numpy as np
import re
import numpy as np
import pandas as pd

import openai
import uvicorn
import nest_asyncio
import json
nltk.download('punkt_tab')

from pyngrok import ngrok
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List
from sentence_transformers import SentenceTransformer
from langchain_milvus import Milvus
from langchain.docstore.document import Document
from nltk.tokenize import sent_tokenize
from google.colab import userdata
from langchain.schema import Document
from langchain.vectorstores import Milvus
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_milvus import Milvus
from langchain.docstore.document import Document
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

from sklearn.metrics import mean_squared_error, roc_auc_score
from math import sqrt
from langchain.embeddings import HuggingFaceEmbeddings
from groq import Groq


from fuzzywuzzy import fuzz
from sentence_transformers import SentenceTransformer

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
def extract_request_type(subject, request_types):
      for rtype in request_types:
          if rtype.lower() in subject.lower():
              return rtype
      return "Unknown"

In [6]:
save_attachments=True
attachment_dir = "/content/drive/My Drive/attachments"

In [7]:

def parseEmail(msg):
    sender = msg['From']
    subject = msg['Subject']
    body = None
    attachment_data_list = []
    email_data = []
    # Request types to search for
    request_types = ["Adjustment", "AU transfer", "Closing Notice", "Commitment Change",
                 "Fee payment", "Money Movement-Inbound", "Money Movement-Outbound"]
#  Create an empty list to store email details


    # Extract email body (text/plain preferred)
    if msg.is_multipart():
        for part in msg.walk():
            content_type = part.get_content_type()
            disposition = part.get_content_disposition()

            # Extract plain text body
            if content_type == "text/plain" and not disposition:
                body = part.get_payload(decode=True).decode(part.get_content_charset(), errors="ignore")

            # Process attachments
            if disposition in ["attachment", "inline"]:
                filename = part.get_filename()
                attachment_data = part.get_payload(decode=True)

                # Save attachment if required
                if save_attachments and filename:
                    if not os.path.exists(attachment_dir):
                        os.makedirs(attachment_dir)

                    file_path = os.path.join(attachment_dir, filename)
                    with open(file_path, 'wb') as f:
                        f.write(attachment_data)

                # Store attachment content in a dictionary
                attachment_data_list.append({
                    "filename": filename,
                    "content": attachment_data.decode(errors="ignore") if attachment_data else None
                })

    else:
        body = msg.get_payload(decode=True).decode(msg.get_content_charset(), errors="ignore")

    request_type = extract_request_type(msg["Subject"], request_types)
    # Create DataFrame

    return {
        "sender": sender,
        "subject": subject,
        "body": body,
        "attachment_count": len(attachment_data_list),
        "attachments": attachment_data_list,
        "Request Type": request_type
        }

In [8]:
def loademails():
  # Path to the directory containing the .eml files
  # eml_directory = "path/to/your/eml/files"
  # List all .eml files
  eml_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.eml')]

  # Function to extract request type from the subject




  save_attachments=True
  attachment_dir = "/content/drive/My Drive/attachments"

  # Create an empty list to store email details
  emaildata = []
  email_data_list = []

  # Loop through all .eml files
  for eml_file in eml_files:
      with open(eml_file, "rb") as f:

        msg = BytesParser(policy=policy.default).parse(f)

      emaildata = parseEmail(msg)
      email_data_list.append(emaildata)
  # Create a DataFrame
  email_df = pd.DataFrame(email_data_list)


  print(email_df.head())
  return email_df


In [9]:
email_df = loademails()

                   sender                       subject  \
0  admin@citizensbank.com  Loan Adjustment Notification   
1  noreply@wellsfargo.com      AU Transfer Confirmation   
2         support@boa.com         Closing Notice Issued   
3    noreply@citibank.com       Commitment Change Alert   
4      billing@usbank.com      Fee Payment Due Reminder   

                                                body  attachment_count  \
0  Dear Team,\n\nPlease be advised that the loan ...                 1   
1  Dear AU Team,\n\nThe AU transfer request for C...                 0   
2  Hello Team,\n\nThis is to inform you that the ...                 0   
3  Dear Customer,\n\nYour commitment for the term...                 0   
4  Dear Finance Team,\n\nA payment of $50,000 is ...                 0   

                                         attachments       Request Type  
0  [{'filename': 'Loan_Adjustment_Email.pdf', 'co...         Adjustment  
1                                                 []

In [10]:
# BERT for email embeddings (Duplicate Detection)
model = SentenceTransformer("all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
def get_email_embedding(email, subject_weight=0.2, body_weight=0.8):
    """
    Generates a weighted embedding using both subject and body.
    Args:
        email (dict): {'subject': '...', 'body': '...'}
        subject_weight (float): Weight for the subject embedding.
        body_weight (float): Weight for the body embedding.
    Returns:
        np.array: Weighted embedding vector.
    """
    subject_embedding = model.encode(email["subject"], convert_to_numpy=True)
    body_embedding = model.encode(email["body"], convert_to_numpy=True)

    # Combine subject & body embeddings with weights
    return (subject_weight * subject_embedding) + (body_weight * body_embedding)


In [12]:
def is_subject_similar(new_subject, existing_subject, threshold=80):
    """
    Checks if the email subject is similar using fuzzy matching.
    """
    return fuzz.partial_ratio(new_subject, existing_subject) > threshold

In [13]:
def storeembeddings():


  #email_df = pd.DataFrame(email_list)
  email_df_dup = email_df[['subject','sender','body']]
  email_df_dup['body'] = email_df_dup['body'].str.replace('\n', ' ', regex=False)

  #stored_embeddings = np.array([get_email_embedding(email) for email in email_list], dtype=np.float32)
  stored_embeddings = np.array([get_email_embedding(email_df_dup.iloc[i].to_dict()) for i in range(len(email_df_dup))], dtype=np.float32)
  # Create FAISS Index
  d = stored_embeddings.shape[1]  # Embedding dimension
  index = faiss.IndexFlatL2(d)
  index.add(stored_embeddings)

  print(f"✅ FAISS index built with {index.ntotal} email embeddings.")
  return index,email_df_dup


In [14]:
totalindex = storeembeddings()
index_faiss = totalindex[0]
email_df_dup = totalindex[1]

<ipython-input-13-6e67d5abf760>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  email_df_dup['body'] = email_df_dup['body'].str.replace('\n', ' ', regex=False)


✅ FAISS index built with 367 email embeddings.


In [51]:
def check_duplicate_reply(new_email,emaildf,index, threshold=0.85):
    """
    Checks if a new email is a duplicate reply from the same sender.
    """

    new_embedding = get_email_embedding(new_email).reshape(1, -1)

    # Search FAISS index for similar embeddings
    D, I = index.search(new_embedding, k=1)
    similarity = 1 - D[0][0]  # Convert L2 distance to similarity

    for idx in I[0]:  # Iterate through closest matches
        matched_email = email_df_dup.iloc[idx]

        # Check if the sender is the same
        if matched_email["sender"] == new_email["sender"]:
            if similarity >= threshold:
                match_condition = "Similarity Threshold"
                print(f" Duplicate reply detected (similarity: {similarity:.2f}) from same sender!")
            elif is_subject_similar(new_email["subject"], matched_email["subject"]):
                match_condition = "Subject Similarity"
                print(f" Possible duplicate reply detected (subject match) from {new_email['sender']}")
            else:
                match_condition = "Same Sender"
                print(f" same sender from {new_email['sender']}")
            duplicate_email_content = {
                "subject": matched_email["subject"],
                "body": matched_email["body"],
                "sender": matched_email["sender"],
                "similarity": similarity
            }
            return {
                "is_duplicate": True,
                "similarity": similarity,
                "duplicate_email": duplicate_email_content,
                "match_condition": match_condition
            }

    print(" No duplicate reply detected.")
    return {
        "is_duplicate": False,
        "similarity": similarity,
        "duplicate_email": None,
        "match_condition": None
    }


In [16]:
def decode_and_extract_email(base64_eml, save_attachments=True, attachment_dir="attachments"):
    """
    Decodes a base64-encoded .eml file and extracts sender, subject, body, and attachments.
    Args:
        base64_eml (str): Base64-encoded email content.
        save_attachments (bool): If True, saves attachments as files.
        attachment_dir (str): Directory to save attachments.
    Returns:
        pd.DataFrame: DataFrame containing extracted email details.
    """
    # Decode base64 to raw email bytes
    eml_bytes = base64.b64decode(base64_eml).decode('utf-8')

    # Parse the email
    msg = BytesParser(policy=policy.default).parse(BytesIO(eml_bytes.encode()))

    emaildatanew = parseEmail(msg)

    #return pd.DataFrame([emaildatanew])
    return emaildatanew


In [17]:
def extract_keywords(text):
    """
    Extracts keywords from the text using simple heuristics.
    """
    words = re.findall(r'\b\w+\b', text)
    keywords = [word for word in words if len(word) > 3]  # Words longer than 3 characters
    return list(set(keywords))

In [18]:
def generate_metadata(eamil_content, request_type):
    """
    Generates metadata for a given title and passage.
    """
    return {
        "request_type": request_type,
        "keywords": extract_keywords(eamil_content),
    }

In [19]:



  #break

In [20]:
def getEmbeddingsAndStore(all_emails):

    documents = all_emails
    print("No.of documents: ",len(documents))
    # Step 2: Initialize Milvus Vector Database
    #print('0')
    #embedding_model = HuggingFaceEmbeddings(model_name="BAAI/LLM-Embedder")
    embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en")
    #print('1')
    # Index parameters
    index_params = {
        "metric_type": "L2",  # Metric for similarity, use "IP" for inner product if needed
        "index_type": "IVF_FLAT",  # Index type (supported in local mode: FLAT, IVF_FLAT, AUTOINDEX)
        "params": {"nlist": 128},  # Parameter specific to the chosen index type
    }
    vectorstore = Milvus.from_documents(  # or Zilliz.from_documents
        documents=documents,
        embedding=embedding_model,
        connection_args={
            "uri": "./milvus_rag_emails.db",
        },
        index_params=index_params,  # Add index parameters here
        drop_old=False,  # Drop the old Milvus collection if it exists
    )

    print("Vector store created successfully!")
    return vectorstore

In [21]:
def bulildvectorstore():
  all_emails = []
  for index, row in email_df.iterrows():
    eamil_content = row['body']
    request_type = row['Request Type']
    metadata = generate_metadata(eamil_content, request_type)
    metadata['keywords'] = json.dumps(metadata['keywords'])
    all_emails.append(Document(page_content=eamil_content, metadata=metadata))
  vectorstore = getEmbeddingsAndStore(all_emails)
  return vectorstore

In [22]:
vectorstore = bulildvectorstore()

No.of documents:  367


<ipython-input-20-55413638f024>:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.3k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Vector store created successfully!


In [23]:
def loadMilvusffromColab(file_path):

  connection_args = {
      "uri": file_path
  }

  # Initialize the embedding model (replace OpenAIEmbeddings with the one you used)
  embedding_model = HuggingFaceEmbeddings(model_name="BAAI/LLM-Embedder")
  #embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en")
  index_params = {
          "metric_type": "L2",  # Metric for similarity, use "IP" for inner product if needed
          "index_type": "IVF_FLAT",  # Index type (supported in local mode: FLAT, IVF_FLAT, AUTOINDEX)
          "params": {"nlist": 128},  # Parameter specific to the chosen index type
      }
  # Load the vector store
  vectorstore = Milvus(
      embedding_function=embedding_model,
      connection_args=connection_args,
      index_params=index_params,  # Add index parameters here
      drop_old=True,  # Drop the old Milvus collection if it exists
  )

  # Now `vectorstore` is loaded and can be used for search or other operations
  print("Vector store loaded from colab successfully!")

  return vectorstore;


In [24]:
# Step 3: Define the Query and Retrieval Function
def retrieve_top_k(query, vectorstore, top_k,printStatements=True):
    """
    Retrieve the top-k relevant documents using Milvus and the MMR retriever.
    """
    #retriever = vectorstore.as_retriever(
    #search_type="mmr",
    #search_kwargs={"k": 10, "lambda_mult": 0.3}  # Adjust `lambda_mult` for relevance-diversity balance
    #)
    #retrieved_docs = retriever.get_top_k_docs(query, k=10)
    # Combine results from both retrievers
    # retriever_sparse = vectorstore.as_retriever(search_type="mmr")
    # retriever_dense = vectorstore.as_retriever(search_type="similarity")
    # sparse_results = retriever_sparse.get_top_k_docs(query,10)
    # dense_results = retriever_dense.get_top_k_docs(query,10)

    # Merge results, prioritize removing duplicates
    #relevant_docs = remove_duplicates(sparse_results + dense_results)
    #retrieved_docs = remove_duplicates(retrieved_docs)
    retriever = vectorstore.as_retriever()
    #retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": top_k, "fetch_k": 8})
    #retriever = vectorstore.as_retriever()
    #retriever = vectorstore.as_retriever(search_type="mmr")
    # retriever = vectorstore.as_retriever(
    #     search_type="similarity",
    #     search_kwargs={
    #         "k": top_k,
    #         "score_threshold": 0.5,  # Adjust this threshold based on your needs
    #         "fetch_k": top_k * 4  # Fetch more candidates for better diversity
    #     }
    # )
    #relevant_docs = retriever.get_relevant_documents(query)
    relevant_docs = retriever.invoke(query)
    #relevant_docs = remove_duplicates(relevant_docs)
    if printStatements:
      print("Relevant Docs length: ",len(relevant_docs))
      print("Relevant Docs: ",relevant_docs)
    # Return only the top_k documents or all available if less than k
    #relevant_docs =  relevant_docs[:top_k] if len(relevant_docs) > top_k else relevant_docs
    #enable it back print("relevant_docs Documents: ",relevant_docs)
    return relevant_docs

In [ ]:

eamil2Cat = email_df.iloc[169]
eamil2Cat_cont = eamil2Cat['body']
eamil2Cat_type = eamil2Cat['Request Type']
print(eamil2Cat_cont)
print(eamil2Cat_type)

relevent_emails = retrieve_top_k(eamil2Cat_cont, vectorstore, 2)

In [26]:
def generateuniquerequesttypes():
  unique_request_types = list(set(email.metadata['request_type'] for email in relevent_emails))

  #request_types = [email.metadata['request_type'] for email in relevent_emails]
  request_type_list = ", ".join(unique_request_types)
  print(request_type_list)
  return request_type_list

In [27]:
request_type_list = generateuniquerequesttypes()

Fee payment


In [28]:
openai_api_key = userdata.get('GROQ_KEY')
# initialize client for Groq
client = Groq(api_key = openai_api_key)

In [29]:
## New prompt andwhich does not require post processing
def createPrompt(question,uniquerequesttypes):
    prompt = f'''You are an expert in classifying commercial loan lending emails. Your task is to identify the request type of the following email based on the provided request types.Also generate a sub request type.

### **Request Types:**
{uniquerequesttypes}

### **Email Content:**
{question}

### **Instructions:**
- Analyze the email content and classify it into one of the provided `request_type` categories.
- Select the most relevant `request_type` based on the context and language of the email.
- Provide a **confidence score** (between 0 and 1) indicating how confident you are in the classification.
- Explain why you classified the email into that `request_type`.

### Important RUles
 1. **Do not include any introductory phrases or text** like "Here is the response", "Response:", or "The answer is".
 2. **Do not add any explanations, formatting, or extra text before or after the JSON.**
 3. The output must **strictly** start with `{{` and end with `}}`.

### **Output Format:**
- Return the result in JSON format with the following fields:
    - `request_type`: The classified request type.
    - `confidence_score`: A float between 0 and 1 indicating confidence.
    - `explanation`: A brief explanation of why you classified the email into that `request_type`.
    - `sub_request_type`: The sub request type based on the request type
  Begin your response with `{{` and output **only the JSON object**.
    '''
    return prompt


In [30]:
def getResponseForQuestion(question,request_type_list,vectorstore, printStatements=True):

    #question = ragbench_all[datasetName]['train'][questionIndex]['question']
    #print("Question: ",question)

    #top_k_docs = retrieve_top_k(question,vectorstore,top_k,printStatements)
    #newchange
    #rerankingdocs = reranking(question,top_k_docs,datasetName,100,skipChunking,printStatements)
    #print("Reranking docs: ",rerankingdocs)
    #context = "\n".join([doc.page_content for doc,score in rerankingdocs])
    #context = "\n".join([doc.page_content for doc in rerankingdocs])



    #newchange
    #repackDoc = repackDocument(context)
    #print("Context after Repacking: ",repackDoc)

    #context,sentence_counter = build_context_from_sentencesList(question,rerankingdocs,printStatements)
    #if printStatements:
     # print("Context after assigning sentence keys: ",context)


    #total_sentences = sum(len(sentences) for sentences in context.values())

    chat_completion = client.chat.completions.create(

        messages=[
            {
                "role": "system",
                "content": createPrompt(question,request_type_list)
                #"content": createPromptForFinQA(context, question)
            }
        ],
        model="llama3-70b-8192",
        #model="llama3-8b-8192",
        temperature=0.5,
        max_tokens=5000,
        top_p=1,
        stop=None,
        stream=False,
    )

    try:
        json_output = json.loads(chat_completion.choices[0].message.content)
    except json.JSONDecodeError:
        # Handle invalid JSON output by wrapping it
        json_output = {
            "request_type": "Unknown",
            "confidence_score": 0.0,
            "explanation": "Failed to parse model output into valid JSON format."
          }

    return json_output




In [53]:
base64_eml ="RnJvbTogYWxlcnRzQGJhbmtjb3JwLmNvbQpUbzogdXNlckBmaW5hbmNlY29ycC5jb20KU3ViamVjdDogTG9hbiBBcHByb3ZhbCBVcGRhdGUgLSBSZWYgOTg3NjUKTUlNRS1WZXJzaW9uOiAxLjAKQ29udGVudC1UeXBlOiBtdWx0aXBhcnQvbWl4ZWQ7IGJvdW5kYXJ5PSI9PT09PT09PT09PT09PT01ODk4NTIwMDI2ODA0MTQxOTM0PT0iCgotLT09PT09PT09PT09PT09PTU4OTg1MjAwMjY4MDQxNDE5MzQ9PQpDb250ZW50LVR5cGU6IHRleHQvcGxhaW47IGNoYXJzZXQ9InV0Zi04IgpDb250ZW50LVRyYW5zZmVyLUVuY29kaW5nOiA3Yml0CgoKICAgIApEZWFyIEN1c3RvbWVyLAoKQ29uZ3JhdHVsYXRpb25zISBZb3VyIGxvYW4gcmVxdWVzdCBoYXMgYmVlbiBhcHByb3ZlZC4gUGxlYXNlIGZpbmQgdGhlIGF0dGFjaGVkIGFwcHJvdmFsIGxldHRlciBmb3IgbG9hbiByZWZlcmVuY2UgOTg3NjUuCgpMb2FuIEFtb3VudDogJDEsMjAwLDAwMApJbnRlcmVzdCBSYXRlOiAzLjklCkVmZmVjdGl2ZSBEYXRlOiAyMDI1LTA5LTE1CgpJZiB5b3UgaGF2ZSBhbnkgcXVlc3Rpb25zLCBmZWVsIGZyZWUgdG8gY29udGFjdCB1cy4KCkJlc3QgUmVnYXJkcywKQmFua0NvcnAgQWxlcnRzIFRlYW0KCgotLT09PT09PT09PT09PT09PTU4OTg1MjAwMjY4MDQxNDE5MzQ9PQpDb250ZW50LVR5cGU6IGFwcGxpY2F0aW9uL3BkZgpDb250ZW50LVRyYW5zZmVyLUVuY29kaW5nOiBiYXNlNjQKQ29udGVudC1EaXNwb3NpdGlvbjogYXR0YWNobWVudDsgZmlsZW5hbWU9IkF0dGFjaG1lbnRfNDkxX3BkZi5wZGYiCk1JTUUtVmVyc2lvbjogMS4wCgpKVkJFUmkweExqTUtKWk9NaTU0Z1VtVndiM0owVEdGaUlFZGxibVZ5WVhSbFpDQlFSRVlnWkc5amRXMWxiblFnYUhSMGNEb3ZMM2QzCmR5NXlaWEJ2Y25Sc1lXSXVZMjl0Q2pFZ01DQnZZbW9LUER3S0wwWXhJRElnTUNCU0NqNCtDbVZ1Wkc5aWFnb3lJREFnYjJKcUNqdzgKQ2k5Q1lYTmxSbTl1ZENBdlNHVnNkbVYwYVdOaElDOUZibU52WkdsdVp5QXZWMmx1UVc1emFVVnVZMjlrYVc1bklDOU9ZVzFsSUM5RwpNU0F2VTNWaWRIbHdaU0F2Vkhsd1pURWdMMVI1Y0dVZ0wwWnZiblFLUGo0S1pXNWtiMkpxQ2pNZ01DQnZZbW9LUER3S0wwTnZiblJsCmJuUnpJRGNnTUNCU0lDOU5aV1JwWVVKdmVDQmJJREFnTUNBMk1USWdOemt5SUYwZ0wxQmhjbVZ1ZENBMklEQWdVaUF2VW1WemIzVnkKWTJWeklEdzhDaTlHYjI1MElERWdNQ0JTSUM5UWNtOWpVMlYwSUZzZ0wxQkVSaUF2VkdWNGRDQXZTVzFoWjJWQ0lDOUpiV0ZuWlVNZwpMMGx0WVdkbFNTQmRDajQrSUM5U2IzUmhkR1VnTUNBdlZISmhibk1nUER3S0NqNCtJQW9nSUM5VWVYQmxJQzlRWVdkbENqNCtDbVZ1ClpHOWlhZ28wSURBZ2IySnFDanc4Q2k5UVlXZGxUVzlrWlNBdlZYTmxUbTl1WlNBdlVHRm5aWE1nTmlBd0lGSWdMMVI1Y0dVZ0wwTmgKZEdGc2IyY0tQajRLWlc1a2IySnFDalVnTUNCdlltb0tQRHdLTDBGMWRHaHZjaUFvWVc1dmJubHRiM1Z6S1NBdlEzSmxZWFJwYjI1RQpZWFJsSUNoRU9qSXdNalV3TXpJME1EVXlOREl3S3pBd0p6QXdKeWtnTDBOeVpXRjBiM0lnS0ZKbGNHOXlkRXhoWWlCUVJFWWdUR2xpCmNtRnllU0F0SUhkM2R5NXlaWEJ2Y25Sc1lXSXVZMjl0S1NBdlMyVjVkMjl5WkhNZ0tDa2dMMDF2WkVSaGRHVWdLRVE2TWpBeU5UQXoKTWpRd05USTBNakFyTURBbk1EQW5LU0F2VUhKdlpIVmpaWElnS0ZKbGNHOXlkRXhoWWlCUVJFWWdUR2xpY21GeWVTQXRJSGQzZHk1eQpaWEJ2Y25Sc1lXSXVZMjl0S1NBS0lDQXZVM1ZpYW1WamRDQW9kVzV6Y0dWamFXWnBaV1FwSUM5VWFYUnNaU0FvZFc1MGFYUnNaV1FwCklDOVVjbUZ3Y0dWa0lDOUdZV3h6WlFvK1BncGxibVJ2WW1vS05pQXdJRzlpYWdvOFBBb3ZRMjkxYm5RZ01TQXZTMmxrY3lCYklETWcKTUNCU0lGMGdMMVI1Y0dVZ0wxQmhaMlZ6Q2o0K0NtVnVaRzlpYWdvM0lEQWdiMkpxQ2p3OENpOUdhV3gwWlhJZ1d5QXZRVk5EU1VrNApOVVJsWTI5a1pTQXZSbXhoZEdWRVpXTnZaR1VnWFNBdlRHVnVaM1JvSURJeE5nbytQZ3B6ZEhKbFlXMEtSMkZ6TXlwZkpGbGpXaVpDClBUZFFPbHR2UnpRdVlIVnZiVDh0UVY4MlNtQW1JVTRpVzNBL1lXY29aVWRwY1NWT0wxSlVhbWxoT2lVd0pVUTRiMHdrVG5Va2RXcGYKUzFOa1AxczVRMHBnY0c5amFXd3liRzl0WTJnMkxGRnphVHcxVkRzM1RGaFpTbFV5T1cxTE4xWlJOVGRVVFVCc1FTOWFKeWxFWjBkeApSalZhVWlwV2NXMVlWR2s4TDFGVU9sSkNXRUJpVW1BbU1HdE5JM05iUVR4aVBsWkFSbWx2V0d4SFJXQWxXMWxWWUU0MmJXOHpjbDVVClJGQmlNMGRGUjJZOVZuTmlYbTVrS0U0M1hUcGlZQ2crWGlGTFBuUlphMzQrWlc1a2MzUnlaV0Z0Q21WdVpHOWlhZ3A0Y21WbUNqQWcKT0Fvd01EQXdNREF3TURBd0lEWTFOVE0xSUdZZ0NqQXdNREF3TURBd056TWdNREF3TURBZ2JpQUtNREF3TURBd01ERXdOQ0F3TURBdwpNQ0J1SUFvd01EQXdNREF3TWpFeElEQXdNREF3SUc0Z0NqQXdNREF3TURBME1EUWdNREF3TURBZ2JpQUtNREF3TURBd01EUTNNaUF3Ck1EQXdNQ0J1SUFvd01EQXdNREF3TnpZNElEQXdNREF3SUc0Z0NqQXdNREF3TURBNE1qY2dNREF3TURBZ2JpQUtkSEpoYVd4bGNnbzgKUEFvdlNVUWdDbHM4T0Rnd01qbGtaakV4WmpSa1pEUTNOakZpWlRrelpUVTJOV0UzTkRrME5tTStQRGc0TURJNVpHWXhNV1kwWkdRMApOell4WW1VNU0yVTFOalZoTnpRNU5EWmpQbDBLSlNCU1pYQnZjblJNWVdJZ1oyVnVaWEpoZEdWa0lGQkVSaUJrYjJOMWJXVnVkQ0F0CkxTQmthV2RsYzNRZ0tHaDBkSEE2THk5M2QzY3VjbVZ3YjNKMGJHRmlMbU52YlNrS0NpOUpibVp2SURVZ01DQlNDaTlTYjI5MElEUWcKTUNCU0NpOVRhWHBsSURnS1BqNEtjM1JoY25SNGNtVm1DakV4TXpNS0pTVkZUMFlLCgotLT09PT09PT09PT09PT09PTU4OTg1MjAwMjY4MDQxNDE5MzQ9PS0tCg=="
#base64_eml = "RnJvbTogc2VydmljZXNAY2l0aXplbnNiYW5rLmNvbQpUbzogdXNlckBmaW5hbmNlY29ycC5jb20KU3ViamVjdDogQ29tbWl0bWVudCBDaGFuZ2UgLSBMb2FuIFJlZiAxNTAzNApNSU1FLVZlcnNpb246IDEuMApDb250ZW50LVR5cGU6IG11bHRpcGFydC9taXhlZDsgYm91bmRhcnk9Ij09PT09PT09PT09PT09PTU4OTg1MjAwMjY4MDQxNDE5MzQ9PSIKCi0tPT09PT09PT09PT09PT09NTg5ODUyMDAyNjgwNDE0MTkzND09CkNvbnRlbnQtVHlwZTogdGV4dC9wbGFpbjsgY2hhcnNldD0idXRmLTgiCkNvbnRlbnQtVHJhbnNmZXItRW5jb2Rpbmc6IDdiaXQKCgogICAgRGVhciB1c2VyLAoKICAgIFRoaXMgaXMgYSBub3RpZmljYXRpb24gcmVnYXJkaW5nIHlvdXIgcmVjZW50IEFVIHRyYW5zZmVyIHJlcXVlc3QuIAogICAgUGxlYXNlIGZpbmQgdGhlIGF0dGFjaGVkIGRvY3VtZW50YXRpb24gcmVsYXRlZCB0byBsb2FuIHJlZmVyZW5jZSA2Mzc3Mi4KCiAgICBMb2FuIEFtb3VudDogJDkwOSw2ODYKICAgIEludGVyZXN0IFJhdGU6IDQuNzUlCiAgICBFZmZlY3RpdmUgRGF0ZTogMjAyNS0wOC0xMgoKICAgIFBsZWFzZSByZWFjaCBvdXQgaWYgeW91IGhhdmUgYW55IHF1ZXN0aW9ucy4KCiAgICBSZWdhcmRzLAogICAgU2VydmljZXMKICAgIAoKLS09PT09PT09PT09PT09PT01ODk4NTIwMDI2ODA0MTQxOTM0PT0KQ29udGVudC1UeXBlOiBhcHBsaWNhdGlvbi9wZGYKQ29udGVudC1UcmFuc2Zlci1FbmNvZGluZzogYmFzZTY0CkNvbnRlbnQtRGlzcG9zaXRpb246IGF0dGFjaG1lbnQ7IGZpbGVuYW1lPSJBdHRhY2htZW50XzQ5MV9wZGYucGRmIgpNSU1FLVZlcnNpb246IDEuMAoKSlZCRVJpMHhMak1LSlpPTWk1NGdVbVZ3YjNKMFRHRmlJRWRsYm1WeVlYUmxaQ0JRUkVZZ1pHOWpkVzFsYm5RZ2FIUjBjRG92TDNkMwpkeTV5WlhCdmNuUnNZV0l1WTI5dENqRWdNQ0J2WW1vS1BEd0tMMFl4SURJZ01DQlNDajQrQ21WdVpHOWlhZ295SURBZ2IySnFDanc4CkNpOUNZWE5sUm05dWRDQXZTR1ZzZG1WMGFXTmhJQzlGYm1OdlpHbHVaeUF2VjJsdVFXNXphVVZ1WTI5a2FXNW5JQzlPWVcxbElDOUcKTVNBdlUzVmlkSGx3WlNBdlZIbHdaVEVnTDFSNWNHVWdMMFp2Ym5RS1BqNEtaVzVrYjJKcUNqTWdNQ0J2WW1vS1BEd0tMME52Ym5SbApiblJ6SURjZ01DQlNJQzlOWldScFlVSnZlQ0JiSURBZ01DQTJNVElnTnpreUlGMGdMMUJoY21WdWRDQTJJREFnVWlBdlVtVnpiM1Z5ClkyVnpJRHc4Q2k5R2IyNTBJREVnTUNCU0lDOVFjbTlqVTJWMElGc2dMMUJFUmlBdlZHVjRkQ0F2U1cxaFoyVkNJQzlKYldGblpVTWcKTDBsdFlXZGxTU0JkQ2o0K0lDOVNiM1JoZEdVZ01DQXZWSEpoYm5NZ1BEd0tDajQrSUFvZ0lDOVVlWEJsSUM5UVlXZGxDajQrQ21WdQpaRzlpYWdvMElEQWdiMkpxQ2p3OENpOVFZV2RsVFc5a1pTQXZWWE5sVG05dVpTQXZVR0ZuWlhNZ05pQXdJRklnTDFSNWNHVWdMME5oCmRHRnNiMmNLUGo0S1pXNWtiMkpxQ2pVZ01DQnZZbW9LUER3S0wwRjFkR2h2Y2lBb1lXNXZibmx0YjNWektTQXZRM0psWVhScGIyNUUKWVhSbElDaEVPakl3TWpVd016STBNRFV5TkRJd0t6QXdKekF3SnlrZ0wwTnlaV0YwYjNJZ0tGSmxjRzl5ZEV4aFlpQlFSRVlnVEdsaQpjbUZ5ZVNBdElIZDNkeTV5WlhCdmNuUnNZV0l1WTI5dEtTQXZTMlY1ZDI5eVpITWdLQ2tnTDAxdlpFUmhkR1VnS0VRNk1qQXlOVEF6Ck1qUXdOVEkwTWpBck1EQW5NREFuS1NBdlVISnZaSFZqWlhJZ0tGSmxjRzl5ZEV4aFlpQlFSRVlnVEdsaWNtRnllU0F0SUhkM2R5NXkKWlhCdmNuUnNZV0l1WTI5dEtTQUtJQ0F2VTNWaWFtVmpkQ0FvZFc1emNHVmphV1pwWldRcElDOVVhWFJzWlNBb2RXNTBhWFJzWldRcApJQzlVY21Gd2NHVmtJQzlHWVd4elpRbytQZ3BsYm1Sdlltb0tOaUF3SUc5aWFnbzhQQW92UTI5MWJuUWdNU0F2UzJsa2N5QmJJRE1nCk1DQlNJRjBnTDFSNWNHVWdMMUJoWjJWekNqNCtDbVZ1Wkc5aWFnbzNJREFnYjJKcUNqdzhDaTlHYVd4MFpYSWdXeUF2UVZORFNVazQKTlVSbFkyOWtaU0F2Um14aGRHVkVaV052WkdVZ1hTQXZUR1Z1WjNSb0lESXhOZ28rUGdwemRISmxZVzBLUjJGek15cGZKRmxqV2laQwpQVGRRT2x0dlJ6UXVZSFZ2YlQ4dFFWODJTbUFtSVU0aVczQS9ZV2NvWlVkcGNTVk9MMUpVYW1saE9pVXdKVVE0YjB3a1RuVWtkV3BmClMxTmtQMXM1UTBwZ2NHOWphV3d5Ykc5dFkyZzJMRkZ6YVR3MVZEczNURmhaU2xVeU9XMUxOMVpSTlRkVVRVQnNRUzlhSnlsRVowZHgKUmpWYVVpcFdjVzFZVkdrOEwxRlVPbEpDV0VCaVVtQW1NR3ROSTNOYlFUeGlQbFpBUm1sdldHeEhSV0FsVzFsVllFNDJiVzh6Y2w1VQpSRkJpTTBkRlIyWTlWbk5pWG01a0tFNDNYVHBpWUNnK1hpRkxQblJaYTM0K1pXNWtjM1J5WldGdENtVnVaRzlpYWdwNGNtVm1DakFnCk9Bb3dNREF3TURBd01EQXdJRFkxTlRNMUlHWWdDakF3TURBd01EQXdOek1nTURBd01EQWdiaUFLTURBd01EQXdNREV3TkNBd01EQXcKTUNCdUlBb3dNREF3TURBd01qRXhJREF3TURBd0lHNGdDakF3TURBd01EQTBNRFFnTURBd01EQWdiaUFLTURBd01EQXdNRFEzTWlBdwpNREF3TUNCdUlBb3dNREF3TURBd056WTRJREF3TURBd0lHNGdDakF3TURBd01EQTRNamNnTURBd01EQWdiaUFLZEhKaGFXeGxjZ284ClBBb3ZTVVFnQ2xzOE9EZ3dNamxrWmpFeFpqUmtaRFEzTmpGaVpUa3paVFUyTldFM05EazBObU0rUERnNE1ESTVaR1l4TVdZMFpHUTAKTnpZeFltVTVNMlUxTmpWaE56UTVORFpqUGwwS0pTQlNaWEJ2Y25STVlXSWdaMlZ1WlhKaGRHVmtJRkJFUmlCa2IyTjFiV1Z1ZENBdApMU0JrYVdkbGMzUWdLR2gwZEhBNkx5OTNkM2N1Y21Wd2IzSjBiR0ZpTG1OdmJTa0tDaTlKYm1adklEVWdNQ0JTQ2k5U2IyOTBJRFFnCk1DQlNDaTlUYVhwbElEZ0tQajRLYzNSaGNuUjRjbVZtQ2pFeE16TUtKU1ZGVDBZSwoKLS09PT09PT09PT09PT09PT01ODk4NTIwMDI2ODA0MTQxOTM0PT0tLQo="
printflag = True
email_df_decoded = decode_and_extract_email(base64_eml)
email_df_decoded = pd.DataFrame([email_df_decoded])
email_df_first = email_df_decoded.iloc[0]
result = check_duplicate_reply(email_df_first,email_df_dup,index_faiss)

if result["is_duplicate"]:
     print(f"🚨 Duplicate Email Found! Matches with:\n{result['duplicate_email']['sender']}")
else:
    email_df = pd.concat([email_df, email_df_decoded], ignore_index=True)
          #total = buildFAISSindex(email_df)
    print("✅ No duplicate email found. Adding to database.")
          #print(f"🔹 FAISS index built with {total} email embeddings.")
        # Call the classify_email function
    result = getResponseForQuestion(email_df_decoded.body,request_type_list,vectorstore,printflag)

 No duplicate reply detected.
✅ No duplicate email found. Adding to database.


In [ ]:
result

In [ ]:


# Define FastAPI app
app = FastAPI()

# Define request model
class EmailRequest(BaseModel):
    email_body: str
# API endpoint to classify emails
@app.post("/classify")
async def classify(request: EmailRequest):
    printflag = True

    try:

        email_df_decoded = decode_and_extract_email(request.email_body)
        email_df_decoded = pd.DataFrame([email_df_decoded])
        email_df_first = email_df_decoded.iloc[0]
        #result = getResponseForQuestion(email_df_decoded.body,request_type_list,vectorstore,printflag)
        dupresult = check_duplicate_reply(email_df_first,email_df_dup,index_faiss)

        if dupresult["is_duplicate"]:
          print(f"🚨 Duplicate Email Found! Matches with:\n{dupresult['similarity']}")
          return {
            "duplicateflag": dupresult["is_duplicate"]
        }
        else:
          #email_df = pd.concat([email_df, email_df_decoded], ignore_index=True)
          #total = buildFAISSindex(email_df)
         # print("✅ No duplicate email found. Adding to database.")
          #print(f"🔹 FAISS index built with {total} email embeddings.")
        # Call the classify_email function



          result = getResponseForQuestion(email_df_decoded.body,request_type_list,vectorstore,printflag)

        return {
            "request_type": result["request_type"],
            "subrequest_type": result["sub_request_type"],
            "confidence_score": result["confidence_score"],
            "reason": result["explanation"],
            "extractedattributes": [],
            "duplicateflag": dupresult["is_duplicate"]
        }

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


ngrok_token = userdata.get('NGROK_TOKEN')
!ngrok authtoken $ngrok_token


# Start the FastAPI server
# Allows execution in Colab
nest_asyncio.apply()



# Use ngrok to expose the FastAPI service
public_url = ngrok.connect(8000).public_url
print(f"Public URL: {public_url}")

# Start FastAPI server
uvicorn.run(app, host="0.0.0.0", port=8000)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: https://20f6-35-186-160-205.ngrok-free.app


INFO:     Started server process [201]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


 No duplicate reply detected.
INFO:     223.184.31.110:0 - "POST /classify HTTP/1.1" 200 OK
